In [42]:
import pandas as pd
import xarray as xr
import math
from datetime import datetime
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
import seaborn as sns
import scipy
import scipy.stats as stats
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose

In [43]:
mur_nc = xr.open_dataset('MUR_patchIds_timeseries_200206-202105.nc')
mur_nc = mur_nc['analysed_sst'].sel(time=slice('2002-01-01', '2020-12-31'))-273.15

<xarray.DataArray 'analysed_sst' (time: 6789, Id: 180)>
array([[24.591095, 24.506622, 24.614288, ..., 27.926483, 27.864563,
        27.95929 ],
       [24.55304 , 24.480103, 24.61731 , ..., 27.933685, 27.736755,
        27.863647],
       [24.419464, 24.386383, 24.475677, ..., 27.97165 , 27.781738,
        27.874664],
       ...,
       [22.298615, 22.180359, 22.446411, ..., 28.404663, 27.588287,
        27.865448],
       [22.311798, 22.304382, 22.502258, ..., 28.455627, 27.99057 ,
        28.036682],
       [22.591461, 22.681366, 22.829834, ..., 28.257477, 28.104279,
        28.090149]], dtype=float32)
Coordinates:
  * Id       (Id) int64 1 2 3 4 5 6 7 8 9 ... 173 174 175 176 177 178 179 180
  * time     (time) datetime64[ns] 2002-06-01T09:00:00 ... 2020-12-31T09:00:00

In [44]:
# NOT INTERPOLATED CRW_SST DATA
mur_nc = xr.open_dataset('CRW_SST_Interpolated_PatchIDs_1985-2020.nc')
mur_nc = mur_nc.sel(time=slice('1993-01-01', '2019-12-31'))
mur_nc = mur_nc['CRW_SST']

# IDENTIFYING NAN IDs
#ref_file = mur_nc
#a = (np.where(np.isnan(ref_file)))
#np.unique(a[0])

<xarray.DataArray 'CRW_SST' (Id: 180, time: 9861)>
[1774980 values with dtype=float64]
Coordinates:
  * time     (time) datetime64[ns] 1993-01-01T12:00:00 ... 2019-12-31T12:00:00
    crs      int32 ...
  * Id       (Id) int64 1 2 3 4 5 6 7 8 9 ... 173 174 175 176 177 178 179 180

In [45]:
mhw_max_intensity = {}
mhw_years_occurrence = {}
mhw_cumulative_intensity = {}
mhw_number_occurrences = {}
mhw_max_duration = {}
mhw_peak_timestamp = {}
mhw_category = {}
delta = {}
mhws = {}
IDs = []

for i in range(1,181):
    IDs.append(i)

for k in range(1, 181):
    #if k == 84 or k == 89 or k == 111 or k == 125 or k == 129: # CONDITION EXCLUDING NAN PATCH IDS
        #k = k+1
    sst_reef = mur_nc.sel(Id=k)
    df = sst_reef.to_dataframe()
    sst_reef = df['CRW_SST']

    decomposition = seasonal_decompose(sst_reef, period = 365) # PERIOD = 365 POIS OS DADOS SÃO DIÁRIOS
    trend = decomposition.trend # COMPONENTE TENDÊNCIA
    seasonal = decomposition.seasonal # COMPONENTE SAZONAL
    residual = decomposition.resid # COMPONENTE RESIDUCAL

    sst_reef_normalized = sst_reef - seasonal
    sst_reef_mean = np.average(sst_reef_normalized)
    sst_reef_anomalies = sst_reef_normalized - sst_reef_mean
    sst_reef_anomalies[sst_reef_anomalies < 0] = 0

    # IDENTIFICANDO AS MHWS
    heatwaves = np.where(sst_reef_normalized > np.percentile(sst_reef_normalized, 90))
    heatwaves = np.asarray(heatwaves)
    heatwaves = heatwaves.tolist()
    heatwaves = heatwaves[0]
    def consecutive(data, stepsize=1):
        return np.split(data, np.where(np.diff(data) > 2)[0]+1)

    consecutive = consecutive(heatwaves)

    # CALCULANDO O DELTA_T
    delta_t = np.percentile(sst_reef_normalized, 90) - sst_reef_mean

    # EXCLUINDO ARRAYS COM MENOS DE 5 DIAS DE DURAÇÃO
    mhws = []
    for i in range(len(consecutive)):
        if len(consecutive[i]) >= 5:
            mhws.append(consecutive[i])

    # CONTANDO A DURAÇÃO MÁXIMA DE MHW PRA CADA ID
    duration_mhws = []
    for i in range(len(mhws)):
        duration = len(mhws[i])
        duration_mhws.append(duration)
    max_duration_mhws = np.max(duration_mhws)
    
    # CONTANDO O NÚMERO DE MHDS PRA CADA ID
    number_mhws = len(mhws)

    # CALCULANDO A INTENSIDADE MÁXIMA DE MHW EM CADA ID E INTENSIDADE CUMULATIVA
    intensity = []
    for i in range(len(mhws)):
        a = mhws[i]
        for j in range(len(a)):
            anomalie = sst_reef_normalized.iloc[mhws[i][j]] - sst_reef_mean
            intensity.append(anomalie)
    intensity_max = np.max(intensity)
    cumulative =  (np.sum(intensity))
    
    # VERIFICANDO A DATA DO PICO MÁXIMO DE INTENSIDADE
    date = np.where(sst_reef_anomalies == intensity_max)
    tpeak = sst_reef_anomalies.index[date]
    tpeak = tpeak.year.values
    tpeak = tpeak[0]
    
    # CONTABILIZANDO A OCORRÊNCIA DE ONDAS DE CALOR POR CADA ANO
    years = []
    for i in range(len(mhws)):
        a = mhws[i][0]
        ano = sst_reef_anomalies.index[a]
        ano = ano.year
        years.append(ano)
        
    # CATEGORIZANDO A INTENSIDADE MÁXIMA DE ACORDO COM HOBDAY (2018)
    if intensity_max < 2*delta_t:
        category = 'Moderate'
    elif intensity_max >= 2*delta_t and intensity_max < 3*delta_t:
        category = 'Strong'
    elif intensity_max >= 3*delta_t and intensity_max < 4*delta_t:
        category = 'Severe'
    elif intensity_max >= 4*delta_t:
        category = 'Extreme'

    # ENCONTRANDO OS DADOS ACIMA DE 90 PERCENTIL
    sst_reef_percentile = sst_reef_normalized
    dummy_reef = sst_reef_percentile
    sst_reef_percentile[sst_reef_percentile < np.percentile(dummy_reef, 90)] = 0

    # ANOMALIAS DOS 90 PERCENTILS
    sst_reef_percentile_anomalies = sst_reef_percentile - sst_reef_mean
    sst_reef_percentile_anomalies[sst_reef_percentile_anomalies < 0] = 0
    
    #mhws["patchID_{0}".format(i)] = mhws
    mhw_cumulative_intensity[k] = cumulative
    mhw_max_intensity[k] = intensity_max
    mhw_max_duration[k] = max_duration_mhws
    delta[k] = delta_t
    mhw_category[k] = category
    mhw_peak_timestamp[k] = tpeak
    mhw_number_occurrences[k] = number_mhws
    mhw_years_occurrence[k] = years

In [47]:
df = pd.DataFrame(mhw_category.items())
patch_ids_mhws = df
df = pd.DataFrame(mhw_max_intensity.items())
patch_ids_mhws = pd.merge(patch_ids_mhws, df, left_on=0, right_on=0, how='left')
df = pd.DataFrame(mhw_max_duration.items())
patch_ids_mhws = pd.merge(patch_ids_mhws, df, left_on=0, right_on=0, how='left')
df = pd.DataFrame(mhw_peak_timestamp.items())
patch_ids_mhws = pd.merge(patch_ids_mhws, df, left_on=0, right_on=0, how='left')
#patch_ids_mhws.columns = ['Id', 'Category', 'Imax', 'Dmax', 'Ypeak'] # SERIA BOM INCLUIR A DATA DO PICO
df = pd.DataFrame(mhw_number_occurrences.items())
patch_ids_mhws = pd.merge(patch_ids_mhws, df, left_on=0, right_on=0, how='left')
df = pd.DataFrame(mhw_cumulative_intensity.items())
patch_ids_mhws = pd.merge(patch_ids_mhws, df, left_on=0, right_on=0, how='left')
patch_ids_mhws
patch_ids_mhws.columns = ['Id', 'Category', 'Imax', 'Dmax', 'Ypeak', 'nMHWs', 'CumInt']
patch_ids_mhws.to_csv('patch_ids_mhws.csv', header=True, index=False)


patch_ids = pd.read_csv('patch_ids_ecoregions_v3.csv')
patch_ids_mhws = pd.merge(patch_ids_mhws, patch_ids, left_on='Id', right_on='Id', how='left')
del patch_ids_mhws['FID']
del patch_ids_mhws['Region']
patch_ids_mhws
patch_ids_mhws.to_csv('patch_ids_mhws.csv', header=True, index=False)

In [15]:
patch_ids_mhws.to_csv('patch_ids_mhws.csv', header=True, index=False)

In [68]:
patch_ids_mhws.groupby(['Ecoregion'])['nMHWs'].std()

Ecoregion
AMZ      1.763834
EAST     5.472033
FNA      2.516611
NEAST    4.999277
TMV      5.033223
Name: nMHWs, dtype: float64

In [70]:
patch_ids_mhws.groupby(['Ecoregion'])['Dmax'].std()

Ecoregion
AMZ       3.153481
EAST     15.508671
FNA       5.773503
NEAST    26.936428
TMV      19.347696
Name: Dmax, dtype: float64

In [71]:
patch_ids_mhws.groupby(['Ecoregion', 'Ypeak'])['Imax'].std()

Ecoregion  Ypeak
AMZ        1998          NaN
           2010     0.142714
EAST       1996     0.021551
           2010     0.069521
           2019     0.158140
FNA        1996     0.015495
           2009          NaN
NEAST      1994     0.038732
           1996     0.087154
           2019     0.060437
TMV        2009     0.158347
Name: Imax, dtype: float64

In [9]:
patch_ids_mhws.groupby(['Ecoregion'], as_index=False)['Imax'].max()

,Ecoregion,Imax
0,AMZ,1.817550
1,EAST,2.444908
2,FNA,1.290794
3,NEAST,1.736210
4,TMV,2.271210


In [72]:
patch_ids_mhws.groupby(['Ecoregion'])['Imax'].std()

Ecoregion
AMZ      0.137868
EAST     0.196573
FNA      0.013035
NEAST    0.120038
TMV      0.158347
Name: Imax, dtype: float64

In [73]:
patch_ids_mhws.groupby(['Ecoregion'])['CumInt'].std()

Ecoregion
AMZ      15.648768
EAST     74.743284
FNA      13.707949
NEAST    42.057859
TMV       8.536310
Name: CumInt, dtype: float64